In [242]:
%cd ~/comp_mut

/mnt/storage7/gary/comp_mut


In [243]:
%reset -f

In [244]:
import json
from collections import defaultdict, Counter
import argparse
import os
from tqdm import tqdm
import sys
import csv
import pathogenprofiler as pp
import tbprofiler
from csv import DictReader
from collections import Counter
import requests
from contextlib import closing
import re
from python_scripts.utils import *

In [245]:
def get_vars_exclude(vars_exclude_file):

    # URL below is the results of all Fst = 1 variants from https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00817-3
    fst_results_url = 'https://raw.githubusercontent.com/GaryNapier/tb-lineages/main/fst_results_clean_fst_1_for_paper.csv'
    # See https://www.codegrepper.com/code-examples/python/how+to+read+a+csv+file+from+a+url+with+python for pulling data from url
    with closing(requests.get(fst_results_url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        fst_dict = csv_to_dict_multi(f)
    
    lin_specific_variants = []
    for gene in fst_dict:
        if gene in genes:
            for var in fst_dict[gene]:
                lin_specific_variants.append( tuple( [ gene, reformat_mutations(var['aa_pos']) ] ) )

    # Read in variants to be excluded
    vars_exclude = []
    for l in open(vars_exclude_file):
        vars_exclude.append(tuple(l.strip().split(',')))

    # Concat
    vars_exclude = vars_exclude + lin_specific_variants 
    return vars_exclude

In [246]:
# def get_counts(in_dict, ref_dict, data_key):
#     data_dict = {k:[] for k in in_dict.keys()}
#     for mut in in_dict:
#         for samp in in_dict[mut]:
#             data_dict[mut].append(ref_dict[samp][data_key])

#     data_counts = {k:[] for k in in_dict.keys()}
#     for mut in data_dict:
#         data_counts[mut] = dict(Counter(data_dict[mut]))
#     return data_counts

In [247]:
# def get_unique_mutations(in_dict, gene):
#     # Pull all mutations from a certain gene and take unique
#     mutations_set = []
#     for samp in in_dict:
#         mut_list = in_dict[samp]['mutations']
#         mutations_set.append(mut['change'] for mut in mut_list if mut['gene'] == gene)
#     # Convert to flat list 
#     mutations_set = flat_list(mutations_set)
#     # Sort
#     mutations_set.sort()
#     # Make a table (before converting to a set)
#     table = Counter(mutations_set)
#     # Get unique
#     mutations_set = set(mutations_set)
#     return (table, mutations_set)

In [248]:
# def dr_filter(in_dict):

#     # Test the proportion of sensitive DR types to all other DR types
#     # If the majority are sensitive, then 'reject' (0) or 'accept' (1) 
#     # Take a dictionary of DR type counts like this:
#     # {'MDR': 15, 'Pre-MDR': 20, 'Pre-XDR': 8, 'XDR': 1, 'Other': 1}
#     # If only 'Sensitive' in the dict the return accept = 0 
#     # If 'Sensitive' is not in the counts then accept = 1
#     # Else test the proportions

#     is_dict(in_dict)

#     if all('Sensitive' in x for x in in_dict):
#         accept = 0
#         return accept

#     if 'Sensitive' not in in_dict:
#         accept = 1
#         return accept

#     sum_non_sens = sum(in_dict[item] for item in in_dict if item != 'Sensitive')

#     proportion = in_dict['Sensitive'] / (in_dict['Sensitive'] + sum_non_sens)

#     if proportion >= 0.5: 
#         accept = 0 
#     else:
#         accept = 1

#     return accept

In [249]:
# def lin_filter(in_dict):
#     is_dict(in_dict)
#     lin_filter_dict = {}
#     for mutation in in_dict:
#         if len(in_dict[mutation]) > 1:
#             lin_filter_dict[mutation] = 1
#         else:
#             lin_filter_dict[mutation] = 0
#     return lin_filter_dict

In [250]:
# def dst_filter(in_dict):

#     if all(x == 'NA' for x in in_dict.keys()):
#         accept = 1
#         return accept

#     if all(x == '1' for x in in_dict.keys()):
#         accept = 1
#         return accept

#     if all(x == '0' for x in in_dict.keys()):
#         accept = 0
#         return accept

#     # Avoid key errors - add keys if not there and only have combination of '0' & 'NA' or '1' & 'NA'
#     if '0' not in in_dict.keys() and 'NA' in in_dict.keys():
#         in_dict['0'] = 0

#     if '1' not in in_dict.keys() and 'NA' in in_dict.keys():
#         in_dict['1'] = 0

#     is_dict(in_dict)
#     proportion = in_dict['1'] / (in_dict['0'] + in_dict['1'])

#     if proportion < 0.5:
#         accept = 0
#     else:
#         accept = 1

#     return accept

In [251]:
# def invert_tb_dict(in_dict):
#     inv_dict = {}
#     for samp in in_dict:
#         for var in in_dict[samp]['mutations']:
#             if var['change'] not in inv_dict:
#                 inv_dict[var['change']] = [samp]
#             else:
#                 inv_dict[var['change']].append(samp)
#     return inv_dict

In [252]:
# def tb_data(filename, suffix, samps_list, genes, vars_exclude):
#     # Read in all the json data for samples with given genes only (>0.7 freq), non-syn
#     all_data = defaultdict(list)
#     meta_dict = defaultdict(dict)

#     for samp in tqdm(samps_list):
#         # Load file for each samp
#         file = "%s/%s%s" % (filename, samp, suffix)
#         if os.path.isfile(file):
#             # Open the json file for the sample, skip if can't find
#             tmp_data = json.load(open(file))
#             # Remove mixed samps
#             if ";" in tmp_data['sublin']: continue
#             # Get metadata
#             meta_dict[samp] = {
#                 'wgs_id':samp,
#                 # 'inh_dst':meta_dict[samp]['isoniazid'],
#                 'main_lin':tmp_data['main_lin'],
#                 'sublin':tmp_data['sublin'],
#                 # 'country_code':meta_dict[samp]['country_code'],
#                 'drtype':standardise_drtype[tmp_data['drtype']]}

#             # Loop over all variants of interest
#             for var in tmp_data["dr_variants"] + tmp_data["other_variants"]:
#                 if var['gene'] not in genes: continue
#                 if var['freq'] < 0.7: continue
#                 if var['type'] == 'synonymous_variant': continue
#                 # Store key as tuple of gene and mutation and append the sample. Ignore if in exclude list.
#                 key = (var['gene'], var['change'])
#                 if key in vars_exclude: continue
#                 all_data[key].append(samp)

#     # Remove single samples
#     for var in list(all_data):
#         if len(all_data[var]) < 2:
#             del all_data[var]

#     return (all_data, meta_dict)

In [253]:
# def merge_metadata(meta_dict, meta_dict_csv, drug_of_interest):
#     # Merge metadata from metadata csv file and from json data (tb_data() function)
#     # Need country code and DST for drug of interest

#     for samp in meta_dict:
#         meta_dict[samp]['dst'] = meta_dict_csv[samp][drug_of_interest]
#         meta_dict[samp]['country_code'] = meta_dict_csv[samp]['country_code']
#     return meta_dict

In [254]:
# def get_all_counts(all_vars_dict, variant_list, meta_dict):
    
#     # Get number of samples by DR type, lineage and DST for a list of mutations of interest 
    
#     # all_vars_dict:
#     # Dictionary 
#     # Keys = tuple of gene and variant, e.g. "('ahpC', 'c.-51G>A')"
#     # Values = list of samples, e.g. "['ERR1465765','SAMEA3715554','ERR1193661',...etc]"
    
#     # variant_list:
#     # List of tuples of gene-variant pairs, e.g.:
#     # [('ahpC', 'c.-51G>A'),
#     # ('ahpC', 'c.-72C>T'),...etc]
    
#     # meta_dict:
#     # Dictionary of metadata, e.g. 
#     #     {'SAMEA2534433': {'wgs_id': 'SAMEA2534433',
#     #               'main_lin': 'lineage2',
#     #               'sublin': 'lineage2.2.1',
#     #               'drtype': 'Other',
#     #               'dst': '0',
#     #               'country_code': 'af'},
    
#     # Return:
#     # Dict for each variant of interest with keys and gene-variant pairs and values as counts
#     #     {('ahpC', 'c.-51G>A'): {'MDR-TB': 16,
#     #    'Pre-MDR-TB': 18,
#     #    'Pre-XDR-TB': 10,
#     #    'Other': 1},... etc}
    
    
#     # Subset the data to the gene-variants of interest data to just the ahpC unknown
#     variants_samps = {var: all_vars_dict[var] for var in variant_list}
    
#     # ahpc_drtype_counts = get_counts(unknown_ahpc_samps_dict, ahpc_dict, 'drtype')
#     drtype_counts = get_counts(variants_samps, meta_dict, 'drtype')
    
#     # Lineage counts - mutation is only from one (or two) lineages
#     lin_counts = get_counts(variants_samps, meta_dict, 'sublin')
    
#     # Aggregate lin counts
#     for mut in lin_counts:
#         lin_counts[mut] = resolve_lineages(lin_counts[mut])
        
#     # DST counts
#     dst_counts = get_counts(variants_samps, meta_dict, 'dst')
    
#     # Put together
#     out_dict = {'drtype_counts': drtype_counts, 'lin_counts': lin_counts, 'dst_counts': dst_counts}
    
#     return out_dict

In [255]:
def get_counts(meta,samples,col):
    return dict(Counter([meta[s][col] for s in samples]))

In [ ]:
def get_meta_proportion(meta,samples,column,targets):
    tmp = get_counts(meta,samples,column)
    target_count = sum([tmp.get(c,0) for c in targets])
    return round(target_count/sum(tmp.values()), 3)

In [256]:
def filter_vars(variants, mutation2sample, meta_dict, drug_of_interest):

    # Get stats for each variant
    stats_dict = defaultdict(dict)
    variants_passed = set()
    # for var in variants:
    for var in variants:

        # Get proportion or number of samples (in the case of lineage) per potential mutation
        samps = mutation2sample[var]

        if len(samps)<3: continue

        dst_proportion = get_meta_proportion(meta_dict,samps,drug_of_interest,['1'])
        sensitive_geno_proportion = get_meta_proportion(meta_dict,samps,'drtype',['Sensitive'])
        num_lins = len(set(resolve_lineages(get_counts(meta_dict,samps,'sublin'))))
        stats_dict[var] = {'n_samps' : len(samps), 
                           'dst_prop': dst_proportion, 
                           'dr_prop': sensitive_geno_proportion, 
                           'n_lins': num_lins}

        # Filter
        for var in stats_dict:
            if stats_dict[var]['dst_prop'] >= 0.5 and \
            stats_dict[var]['dr_prop'] <= 0.5 and \
            stats_dict[var]['n_lins'] > 1:
                variants_passed.add(var)

    return (variants_passed, stats_dict)

In [257]:
# FILES

# def main(args):

# mutations_file = args.mutations_file
# metadata_file = args.metadata_file
# id_key = args.id_key

# mutations_file = "metadata/novel_ahpc_mutations.txt"
# metadata_file = "../metadata/tb_data_18_02_2021.csv"
# id_key = "wgs_id"
# mutaions_key = "wgs_id" # column name of mutation names e.g. "c.-101A>G"

potential_comp_mut_file = "metadata/ahpc_model_results.csv"
metadata_file = "../metadata/tb_data_18_02_2021.csv"
tbdb_file = "../tbdb/tbdb.csv"
drtypes_file = "../pipeline/db/dr_types.json"
comp_mut_file = '../pipeline/db/compensatory_mutations.csv'
# tbprofiler_results_dir = '/mnt/storage7/jody/tb_ena/tbprofiler/freebayes/results/'
# tbprofiler_results_dir = '/mnt/storage7/jody/tb_ena/tbprofiler/gatk/results'
tbprofiler_results_dir = '/mnt/storage7/jody/tb_ena/tbprofiler/freebayes/results/'
vars_exclude_file = 'metadata/var_exclude_katg_comp_mut.csv'


In [258]:
# VARIABLES

# metadata_id_key = "wgs_id"
suffix = ".results.json"
genes = ('ahpC', 'katG', 'fabG1')
drug_of_interest = 'isoniazid'
comp_mut_genes = ('ahpC')

In [259]:
# READ IN DATA

# Read in ahpc GLM results file
with open(potential_comp_mut_file, 'r') as f:
    potential_comp_mut_dict = csv_to_dict(f)

# Convert to list of tuples
potential_comp_mut_list = [(comp_mut_genes, var) for var in list(potential_comp_mut_dict)]

# Read in metadata
with open(metadata_file) as mf:
    meta_dict = csv_to_dict(mf)

# Pull samples
samples = list(meta_dict.keys())

# Read in tbdb file
with open(tbdb_file, 'r') as f:
    tbdb_dict = csv_to_dict_multi(f)

# Read in DR types from json
standardise_drtype = json.load(open(drtypes_file))

# Get known compensatory mutations of interest
compensatory_mutations = defaultdict(set)
for row in csv.DictReader(open(comp_mut_file)):
    if row['Drug'] != drug_of_interest: continue
    compensatory_mutations[row['Drug']].add((row['Gene'],row['Mutation']))
    
# Read in variants to exclude
vars_exclude = get_vars_exclude(vars_exclude_file)


In [260]:
# Read in all the json data for (samples with) ahpC/katG only (>0.7 freq) and the metadata for those samples

# Load mutation data using ('gene','change') as keys
mutation2sample = defaultdict(set)
sample2mutation = defaultdict(set)
resistance_mutations = defaultdict(set)
for s in tqdm(samples):
    file = "%s/%s%s" % (tbprofiler_results_dir, s, suffix)
    if os.path.isfile(file):
        data = json.load(open(file))
        # Skip mixed samps
        if ';' in data['sublin']: continue

        meta_dict[s]['drtype'] = data['drtype']
        meta_dict[s]['sublin'] = data['sublin']
        
        # MAKE SURE THE FOR LOOP BELOW IS INDENTED IN LINE WITH if os.path.isfile(file):
        # Otherwise adds sample s to mutation2sample etc
    
        for var in data['dr_variants'] + data['other_variants']:
            if var['gene'] not in genes: continue
            if var['freq'] < 0.7: continue
            if var['type']=='synonymous_variant': continue
            if (var['gene'], var['change']) in vars_exclude: continue
            key = (var['gene'],var['change'])
            mutation2sample[key].add(s)
            sample2mutation[s].add(key)
            if "drugs" in var:
                for d in var["drugs"]:
                    if key in compensatory_mutations[d["drug"]]: continue
                    resistance_mutations[d["drug"]].add(key)


100%|██████████| 32735/32735 [00:09<00:00, 3570.22it/s]


In [261]:
len(sample2mutation)

14714

In [262]:
len(mutation2sample)

1109

In [263]:
len(resistance_mutations)

2

In [264]:
# ---------
# TESTING
# ---------

# file = "%s/%s%s" % (tbprofiler_results_dir, 'SAMEA2534433', suffix)
# data = json.load(open(file))

# DR TYPES - how to load?


In [270]:
# Classify potential compensatory mutations and filter 
# GLM is only first step in identifying 'interesting' compensatory mutations
# Need to check against tbprofiler results for each mutation 
# e.g. if the mutation is lineage specific, then filter out

potential_comp_mut_filtered, potential_comp_mut_stats = filter_vars(potential_comp_mut_list, \
                                                                    mutation2sample, \
                                                                    meta_dict, \
                                                                    drug_of_interest)

In [271]:
potential_comp_mut_filtered

{('ahpC', 'c.-51G>A'), ('ahpC', 'c.-72C>T')}

In [273]:
# Add the filtered potential compensatory mutations 
# to the list of known compensatory mutations for the drug of interest

compensatory_mutations[drug_of_interest].update(potential_comp_mut_filtered)

In [274]:
samples[0:10]

['SAMEA2534433',
 'ERR1465765',
 'SAMEA3715554',
 'ERR1465906',
 'ERR1193661',
 'ERR1193674',
 'ERR1193709',
 'ERR1465895',
 'ERR757145',
 'ERR757146']

In [275]:
compensatory_mutations

defaultdict(set,
            {'isoniazid': {('ahpC', 'c.-20C>T'),
              ('ahpC', 'c.-39C>T'),
              ('ahpC', 'c.-44T>A'),
              ('ahpC', 'c.-48G>A'),
              ('ahpC', 'c.-48G>T'),
              ('ahpC', 'c.-49T>G'),
              ('ahpC', 'c.-4A>G'),
              ('ahpC', 'c.-51G>A'),
              ('ahpC', 'c.-51G>T'),
              ('ahpC', 'c.-52C>T'),
              ('ahpC', 'c.-54C>T'),
              ('ahpC', 'c.-57C>T'),
              ('ahpC', 'c.-5G>A'),
              ('ahpC', 'c.-66G>A'),
              ('ahpC', 'c.-72C>T'),
              ('ahpC', 'c.-74G>A'),
              ('ahpC', 'c.-81C>T'),
              ('ahpC', 'p.Asp33Asn'),
              ('ahpC', 'p.Asp73His'),
              ('ahpC', 'p.Glu76Lys'),
              ('ahpC', 'p.Leu191Arg'),
              ('ahpC', 'p.Phe10Ile'),
              ('ahpC', 'p.Pro2Ser'),
              ('ahpC', 'p.Thr5Ile')},
             'ethionamide': set()})

In [276]:
resistance_mutations

defaultdict(set,
            {'isoniazid': {('fabG1', 'c.-15C>T'),
              ('fabG1', 'c.-17G>T'),
              ('fabG1', 'c.-8T>A'),
              ('fabG1', 'c.-8T>C'),
              ('fabG1', 'c.-8T>G'),
              ('katG', 'c.-104_*2462del'),
              ('katG', 'c.-10A>C'),
              ('katG', 'c.-1136_*12019del'),
              ('katG', 'c.-11561_*8301del'),
              ('katG', 'c.-1180_1265del'),
              ('katG', 'c.-13060_*5307del'),
              ('katG', 'c.-15133_*11506del'),
              ('katG', 'c.-15408_*19120del'),
              ('katG', 'c.-1778_*1594del'),
              ('katG', 'c.-2055_*3330del'),
              ('katG', 'c.-2233_*35269del'),
              ('katG', 'c.-2717_47del'),
              ('katG', 'c.-3736_*5689del'),
              ('katG', 'c.-4345_*3617del'),
              ('katG', 'c.-4547_574del'),
              ('katG', 'c.-479_*6638del'),
              ('katG', 'c.-5102_1762del'),
              ('katG', 'c.-5249_79del'),
        

In [277]:
potential_resistance_mutations = set()
for s in tqdm(samples):
    # Get the comp, res and other variants for each sample in the full sample list
    comp_var = [var for var in sample2mutation[s] if var in compensatory_mutations[drug_of_interest]]
    res_var = [var for var in sample2mutation[s] if var in resistance_mutations[drug_of_interest]]
    other_vars = [var for var in sample2mutation[s] if var not in compensatory_mutations[drug_of_interest] and \
                  var not in resistance_mutations[drug_of_interest]]
    # If there is at least one comp variant and there are no (known) resistance variants
    if len(comp_var)>0 and len(res_var)==0:
        # If there are no 'other' variants print the sample and the comp variants
        if len(other_vars)==0:
            print("Sample, comp. variant")
            print(s,comp_var)
        # Store the 'other' vars as potential resistance variants
        for var in other_vars:
            potential_resistance_mutations.add(var)



100%|██████████| 32735/32735 [00:00<00:00, 239320.04it/s]

Sample, comp. variant
ERR2516197 [('ahpC', 'p.Leu191Arg')]
Sample, comp. variant
ERR2516199 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR2179658 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SAMN03246450 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
SAMN03246659 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR1723651 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR6824452 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
SRR6824287 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR8651572 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
ERR216923 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
ERR3148230 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR3148166 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR3148207 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR550722 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR553062 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR553076 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
SAMEA2534034 [('ahpC', 'c.-57C>T')]
Sample, comp. variant
ERR553226 [('ah

In [278]:
potential_resistance_mutations

{('ahpC', 'c.-142G>A'),
 ('ahpC', 'c.-256delT'),
 ('ahpC', 'c.-88G>A'),
 ('ahpC', 'p.Leu191Pro'),
 ('fabG1', 'c.-102G>A'),
 ('fabG1', 'c.-179C>T'),
 ('fabG1', 'c.-223_-222insGC'),
 ('katG', 'c.-1T>C'),
 ('katG', 'c.-634_489del'),
 ('katG',
  'c.1042_1128delNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNA'),
 ('katG', 'c.1120_*453del'),
 ('katG', 'c.1438_1440delGCG'),
 ('katG', 'c.2047_*24266del'),
 ('katG', 'c.2223A>C'),
 ('katG', 'c.2223A>G'),
 ('katG', 'c.680_685delTGATCT'),
 ('katG', 'c.718_*16215del'),
 ('katG', 'c.984_998delGGACAACAGTTTCCT'),
 ('katG', 'p.Ala122Val'),
 ('katG', 'p.Ala162Val'),
 ('katG', 'p.Ala312Glu'),
 ('katG', 'p.Ala312Val'),
 ('katG', 'p.Ala411Asp'),
 ('katG', 'p.Ala476Glu'),
 ('katG', 'p.Ala480Gln'),
 ('katG', 'p.Ala621Asp'),
 ('katG', 'p.Ala93Thr'),
 ('katG', 'p.Arg385Pro'),
 ('katG', 'p.Arg484His'),
 ('katG', 'p.Arg705Trp'),
 ('katG', 'p.Arg78Pro'),
 ('katG', 'p.Asn655Asp'),
 ('katG', 'p.Asp142Asn'),
 ('katG', 'p.Asp142

In [279]:
# Filter the potential resistance variants in the same way as filtering the potential comp. variants

potential_res_mut_filtered, potential_res_mut_stats = filter_vars(potential_resistance_mutations, \
                                         mutation2sample,\
                                         meta_dict,\
                                         drug_of_interest)

print(potential_res_mut_filtered)
print()
print(potential_res_mut_stats)

{('katG', 'c.2223A>G'), ('katG', 'p.Asp419Tyr'), ('katG', 'p.Asn655Asp'), ('katG', 'p.Gln88Pro'), ('katG', 'p.Trp90Arg'), ('katG', 'p.Tyr413Cys'), ('katG', 'p.Trp505*'), ('katG', 'p.Trp161Cys'), ('katG', 'p.Gly299Ser'), ('katG', 'p.Gly169Ser'), ('katG', 'p.Pro232Ser'), ('katG', 'p.Arg484His'), ('katG', 'p.Asp142Gly')}

defaultdict(<class 'dict'>, {('katG', 'p.Asp419Tyr'): {'n_samps': 6, 'dst_prop': 0.667, 'dr_prop': 0.0, 'n_lins': 3}, ('katG', 'p.Trp90Arg'): {'n_samps': 3, 'dst_prop': 1.0, 'dr_prop': 0.0, 'n_lins': 3}, ('ahpC', 'p.Leu191Pro'): {'n_samps': 3, 'dst_prop': 1.0, 'dr_prop': 0.0, 'n_lins': 1}, ('fabG1', 'c.-102G>A'): {'n_samps': 170, 'dst_prop': 0.024, 'dr_prop': 0.665, 'n_lins': 2}, ('katG', 'p.Trp161Cys'): {'n_samps': 5, 'dst_prop': 0.6, 'dr_prop': 0.0, 'n_lins': 2}, ('katG', 'p.Trp191Gly'): {'n_samps': 25, 'dst_prop': 0.44, 'dr_prop': 0.08, 'n_lins': 6}, ('katG', 'p.Gly169Ser'): {'n_samps': 7, 'dst_prop': 0.714, 'dr_prop': 0.0, 'n_lins': 2}, ('katG', 'c.984_998delGGACAACA

In [280]:
# DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG DEBUG 


# # def get_counts(meta,samples,col):
# dict(Counter([meta[s][col] for s in samples]))

# # def get_meta_proportion(meta,samples,column,targets):
# tmp = get_counts(meta,samples,column)
# target_count = sum([tmp.get(c,0) for c in targets])
# round(target_count/sum(tmp.values()), 3)


# def filter_vars(variants, mutation2sample, meta_dict, drug_of_interest):



# Get stats for each variant
# stats_dict = defaultdict(dict)
# variants_passed = set()
# # for var in variants:
# for var in potential_resistance_mutations:

#     # Get proportion or number of samples (in the case of lineage) per potential mutation
#     samps = mutation2sample[var]

#     if len(samps)<3: continue

#     dst_proportion = get_meta_proportion(meta_dict,samps,drug_of_interest,['1'])
#     sensitive_geno_proportion = get_meta_proportion(meta_dict,samps,'drtype',['Sensitive'])
#     num_lins = len(set(resolve_lineages(get_counts(meta_dict,samps,'sublin'))))
#     stats_dict[var] = {'n_samps' : len(samps), 
#                        'dst_prop': dst_proportion, 
#                        'dr_prop': sensitive_geno_proportion, 
#                        'n_lins': num_lins}

#     # Filter
#     for var in stats_dict:
#         if stats_dict[var]['dst_prop'] >= 0.5 and \
#         stats_dict[var]['dr_prop'] <= 0.5 and \
#         stats_dict[var]['n_lins'] > 1:
#             variants_passed.add(var)

# variants_passed
# print(samps)

# get_meta_proportion(meta_dict,samps,'drtype',['Sensitive'])

# get_counts(meta_dict,samps,'drtype')


# dict(Counter([meta[s][col] for s in samples]))


# for s in samps:
#     print(s)
#     meta_dict[s]['drtype']
    
# meta_dict['ERR3275405']
# meta_dict['ERR1633786']


# file = "%s/%s%s" % (tbprofiler_results_dir, 'ERR1633786', suffix)
# data = json.load(open(file))
# data['sublin']


In [281]:
mutation2sample

defaultdict(set,
            {('katG', 'p.Ser315Thr'): {'SAMEA2533860',
              'ERR760845',
              'SRR5067222',
              'SRR5065339',
              'SAMN02584635',
              'ERR067721',
              'ERR3276195',
              'SRR1723511',
              'ERR2516842',
              'ERR2510512',
              'ERR2517076',
              'DRR184609',
              'DRR185101',
              'ERR2677422',
              'ERR234600',
              'ERR2041783',
              'SAMN02584631',
              'SAMN02403467',
              'ERR2516523',
              'ERR038745',
              'ERR553238',
              'ERR2510159',
              'SRR6044999',
              'ERR502508',
              'ERR1873503',
              'ERR2515934',
              'ERR751365',
              'ERR1555053',
              'ERR234612',
              'ERR2516905',
              'SRR1166321',
              'SRR7341677',
              'SRR1710093',
              'ERR757189',
         

In [300]:
# Co-occurrence with fabG1

# potential_res_mut_filtered
# mutation2sample
# sample2mutation
# meta_dict
# drug_of_interest
co_gene = 'fabG1'




co_gene_variants = set()
for var in potential_res_mut_filtered:
    samps = mutation2sample[var]
    
    for samp in samps:
        variants = sample2mutation[samp]
        co_gene_variants = [v for v in variants if v[0]==co_gene]
        print(co_gene_variants)


# [var for var in mutation2sample if var[0] == co_gene]

# co_gene_vars = [var for var in mutation2sample if var[0] == co_gene]

# co_gene_samps = [mutation2sample[var] for var in co_gene_vars]


[]
[]
[]
[]
[]
[('fabG1', 'c.-15C>T')]
[]
[('fabG1', 'c.-8T>C')]
[]
[]
[]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[]
[]
[]
[('fabG1', 'c.-15C>T')]
[]
[]
[('fabG1', 'c.-102G>A')]
[('fabG1', 'c.-8T>C')]
[]
[('fabG1', 'c.-15C>T')]
[]
[('fabG1', 'c.-8T>C')]
[('fabG1', 'c.-15C>T')]
[]
[('fabG1', 'c.-15C>T')]
[]
[]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[]
[]
[]
[]
[]
[]
[('fabG1', 'c.-15C>T')]
[]
[]
[]
[]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[('fabG1', 'c.-15C>T')]
[]
[]
[]
[]
[('fabG1', 'c.-15C>T'), ('fabG1', 'c.-102G>A')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
